In [3]:
import numpy as np

In [4]:
def generate_initial_population(N):
    initial_population = np.random.randint(0, 2, (N, 2, 5))
    return initial_population

def evaluate_fitness(solution):
    x1 = sum([2**(4-i) * solution[0][i] for i in range(5)])
    x2 = sum([2**(4-i) * solution[1][i] for i in range(5)])

    constraint_1 = (100 * x1 + 50 * x2) <= 3000
    constraint_2 = (10 * x1) <= 100
    if constraint_1 and constraint_2:
        fitness_value = 100 * x1 + 40 * x2
    else:
        fitness_value = 0 #제약을 하나라도 어기면 적합도 0점

    return fitness_value

def crossover(solution1, solution2):
    crossover_point1 = np.random.randint(1,4) #x1에 대한 교차 지점
    crossover_point2 = np.random.randint(1,4) #x2에 대한 교차 지점

    child = np.empty((2,5)) # 자식을 빈 배열로 생성

    # 부모 유전자 가져오기
    child[0][:crossover_point1] = solution1[0][:crossover_point1]
    child[0][crossover_point1:] = solution2[0][crossover_point1:]
    child[1][:crossover_point2] = solution1[1][:crossover_point2]
    child[1][:crossover_point2] = solution2[1][:crossover_point2]

    return child

def mutation(child, p):
    for row in range(2):
        for col in range(5):
            if np.random.random() < p:
                child[row, col] = 1 - child[row, col]
    
    return child

In [5]:
num_iter = 10 # 세대 수
N = 20 # 한 세대에 포함되는 해의 개수
N_P = 10 # 부모 개수
mutation_sol_prob = 0.1 # 유전자(해)가 돌연변이일 확률
mutation_gene_prob = 0.2 # 유전 개체가 돌연변이일 확률

In [6]:
current_population = generate_initial_population(N)
best_score = -1 # 지금까지 찾은 최대 적합도 초기화

for _ in range(num_iter - 1):
    # 해 평가 수행
    fitness_value_list = np.array([evaluate_fitness(solution) for solution in current_population])

    # 지금까지 찾은 최대 적합도보다 현 세대에 있는 최대 적합도가 크다면 업데이트
    if fitness_value_list.max() > best_score:
        best_score = fitness_value_list.max()
        best_solution = current_population[fitness_value_list.argmax()]

    # 적합도 기준 상위 N_P 개 해 선정(값이 큰 순으로 정렬하기 위해 -를 붙임)
    parents = current_population[np.argsort(-fitness_value_list)]

    # 새로운 해 집단 정의
    new_population = parents

    # 두 개의 부모를 선택하면서 자식 생성
    for _ in range(N-N_P): # N - N_P는 생성해야되는 자식 개수
        # 부모 선택
        parent_1_idx, parents_2_idx = np.random.choice(N_P, 2, replace= False)
        parent_1 = parents[parent_1_idx]
        parent_2 = parents[parents_2_idx]

        # 자식 생성
        child = crossover(parent_1, parent_2)

        # mutation_sol_prob의 확률로 돌연변이 연산 수행
        if np.random.random() < mutation_sol_prob:
            child = mutation(child, mutation_gene_prob)

        # new_population에 child 추가
        new_population = np.vstack([new_population, child.reshape(1, 2, 5)])

In [7]:
best_score

2240

In [8]:
best_solution

array([[0, 1, 0, 1, 0],
       [1, 1, 1, 1, 1]])